## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aripuana,BR,-9.1667,-60.6333,75.29,45,47,1.77,scattered clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,37.06,70,100,14.97,overcast clouds
2,2,Mount Isa,AU,-20.7333,139.5000,64.17,39,23,11.50,few clouds
3,3,Karaton,ID,-6.3424,106.1045,74.39,93,97,1.72,overcast clouds
4,4,Airai,TL,-8.9266,125.4092,63.32,58,36,3.15,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aripuana,BR,-9.1667,-60.6333,75.29,45,47,1.77,scattered clouds
9,9,Abu Zabad,SD,12.3500,29.2500,78.06,71,99,8.63,overcast clouds
11,11,Korcula,HR,42.9622,17.1369,79.77,73,1,4.81,clear sky
14,14,Alofi,NU,-19.0595,-169.9187,77.04,83,25,20.24,light rain
15,15,Broome,US,42.2506,-75.8330,75.99,62,20,6.91,few clouds
16,16,Bandundu,CD,-3.3167,17.3667,79.92,53,67,5.97,broken clouds
22,22,Tual,ID,-5.6667,132.7500,79.14,88,85,18.99,overcast clouds
26,26,Albany,US,42.6001,-73.9662,79.34,54,22,4.29,few clouds
29,29,Kapaa,US,22.0752,-159.3190,84.52,61,0,11.50,clear sky
30,30,Gat,IL,31.6100,34.7642,81.39,81,0,3.18,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                239
City                   239
Country                239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No Empty Rows but need to create new dataframe clean_df
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                239
City                   239
Country                239
Lat                    239
Lng                    239
Max Temp               239
Humidity               239
Cloudiness             239
Wind Speed             239
Current Description    239
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Aripuana,BR,75.29,scattered clouds,-9.1667,-60.6333,
9,Abu Zabad,SD,78.06,overcast clouds,12.3500,29.2500,
11,Korcula,HR,79.77,clear sky,42.9622,17.1369,
14,Alofi,NU,77.04,light rain,-19.0595,-169.9187,
15,Broome,US,75.99,few clouds,42.2506,-75.8330,
16,Bandundu,CD,79.92,broken clouds,-3.3167,17.3667,
22,Tual,ID,79.14,overcast clouds,-5.6667,132.7500,
26,Albany,US,79.34,few clouds,42.6001,-73.9662,
29,Kapaa,US,84.52,clear sky,22.0752,-159.3190,
30,Gat,IL,81.39,clear sky,31.6100,34.7642,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
            
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Aripuana,BR,75.29,scattered clouds,-9.1667,-60.6333,
9,Abu Zabad,SD,78.06,overcast clouds,12.3500,29.2500,حامد محمد نور
11,Korcula,HR,79.77,clear sky,42.9622,17.1369,Marko Polo Hotel by Aminess
14,Alofi,NU,77.04,light rain,-19.0595,-169.9187,Taloa Heights
15,Broome,US,75.99,few clouds,42.2506,-75.8330,Chenango Valley State Park
16,Bandundu,CD,79.92,broken clouds,-3.3167,17.3667,Coja Color
22,Tual,ID,79.14,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
26,Albany,US,79.34,few clouds,42.6001,-73.9662,
29,Kapaa,US,84.52,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
30,Gat,IL,81.39,clear sky,31.6100,34.7642,OlusHome


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df = clean_hotel_df.replace(r'^s*$', float('NaN'), regex=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Abu Zabad,SD,78.06,overcast clouds,12.3500,29.2500,حامد محمد نور
11,Korcula,HR,79.77,clear sky,42.9622,17.1369,Marko Polo Hotel by Aminess
14,Alofi,NU,77.04,light rain,-19.0595,-169.9187,Taloa Heights
15,Broome,US,75.99,few clouds,42.2506,-75.8330,Chenango Valley State Park
16,Bandundu,CD,79.92,broken clouds,-3.3167,17.3667,Coja Color
22,Tual,ID,79.14,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
29,Kapaa,US,84.52,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
30,Gat,IL,81.39,clear sky,31.6100,34.7642,OlusHome
37,Auki,SB,85.69,broken clouds,-8.7676,160.7034,Auki Motel
38,Gushikawa,JP,83.79,broken clouds,26.3544,127.8686,Kirari Guest House


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))